In [2]:
import random

# from allennlp.predictors.predictor import Predictor
# import allennlp_models.coref
# predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz")
pass
import tensorflow_hub as hub
import tensorflow as tf
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from transformers import AutoTokenizer, AutoModelWithLMHead, pipeline
from transformers import pipeline
import numpy as np
import json
import pickle



# FullTokenizer = bert.bert_tokenization.FullTokenizer
# from tensorflow.keras.models import Model       # Keras is the new high level API for TensorFlow
# import math



AttributeError: module 'tensorflow._api.v2.train' has no attribute 'Optimizer'

In [ ]:
mask_dict = dict()
mask_dict["bert-base-cased"] = "[MASK]"
mask_dict["bert-base-uncased"] = "[MASK]"
mask_dict["distilbert-base-uncased"] = "[MASK]"
mask_dict["distilbert-base-cased"] = "[MASK]"
mask_dict["albert-base-v2"] = "[MASK]"

mask_dict["roberta-base"] = "<mask>"

In [ ]:
THRESHOLD = 0.05
# THRESHOLD = 0.1
# THRESHOLD = 0.15
# THRESHOLD = 0.2
# THRESHOLD = 0.25
# THRESHOLD = 0.3

In [4]:
# model_type = "bert-base-cased"
model_type = "bert-base-uncased"
# model_type = "distilbert-base-uncased"
# model_type = "distilbert-base-cased"
# model_type = "albert-base-v2"
# model_type = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained("models/" + model_type +"/")

model = AutoModelWithLMHead.from_pretrained("models/" + model_type +"/")
unmasker = pipeline('fill-mask', model=model, tokenizer=tokenizer)

I0817 21:08:47.169203 4678790592 configuration_utils.py:262] loading configuration file models/bert-base-uncased/config.json
I0817 21:08:47.171446 4678790592 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0817 21:08:47.172374 4678790592 tokenization_utils_base.py:1167] Model name 'models/bert-base-uncased/' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased

In [5]:
with open('saved_pickles/Exploration/unique_input1_set-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
    unique_input1_set = pickle.load(handle)
    
with open('saved_pickles/Exploration/unique_input1_error_set-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
    unique_input1_error_set = pickle.load(handle)

with open('saved_pickles/Exploration/occupation_pair_error-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
    occupation_pair_error = pickle.load(handle)
    
with open('saved_pickles/Exploration/occupation1_error-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
    occupation1_error = pickle.load(handle)
    
with open('saved_pickles/Exploration/occupation2_error-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
    occupation2_error = pickle.load(handle)  
    
with open('saved_pickles/Exploration/verb_error-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
    verb_error = pickle.load(handle)    
    
with open('saved_pickles/Exploration/action_error-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
    action_error = pickle.load(handle)
    
with open('saved_pickles/Exploration/occupation_pair_count-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
    occupation_pair_count = pickle.load(handle)
    
with open('saved_pickles/Exploration/occupation1_count-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
    occupation1_count = pickle.load(handle)
    
with open('saved_pickles/Exploration/occupation2_count-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
    occupation2_count = pickle.load(handle)  
    
with open('saved_pickles/Exploration/verb_count-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
    verb_count = pickle.load(handle)    
    
with open('saved_pickles/Exploration/action_count-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
    action_count = pickle.load(handle)

In [6]:
verb_action = {}
verb_list_p1 = ['walked', 'rode a bike', 'took a train', 'drove a car', 
                'took a taxi', 'got a lift', 'rode a horse', 
                'travelled by sea', 'took a plane', 'took a helicopter', 
                'flew a plane', 'jogged', 'cycled', 'ran', 
                'rode a motorcycle', 'rode a bicycle'] 

action1 = ['home', 'place of work', 'hometown', 'neighborhood', 
           'house', 'domicile', 'office', 'place of residence', 
           'holiday home', 'birthday party', 'musical concert', 
           'final examination']

for verb in verb_list_p1:
    verb_action[verb] = [action1]

In [7]:
# if model_type == "roberta-base":
#     with open("models/" + model_type +"/vocab.json", "r") as vocab_file:
#         vocab_list = json.load(vocab_file)
# else:
vocab_file = open("models/" + model_type +"/vocab.txt", "r")
vocab_list = vocab_file.readlines()

In [8]:
def get_sorted_dict(D):
    return {k: v for k, v in sorted(D.items(), key=lambda item: item[1], reverse=1)}

In [9]:
def get_error_rate_dict(error_dict, count_dict):
    error_rate_dict = {}
    for key in error_dict:
        error_rate_dict[key] = error_dict[key]/count_dict[key]
    return get_sorted_dict(error_rate_dict)

In [10]:
def get_probability_dict(error_dict, count_dict):
    error_rate_dict = get_error_rate_dict(error_dict, count_dict)
    
    probability_dict = {}
    error_rate_sum = sum(error_rate_dict.values())
    for error_rate in error_rate_dict:
        probability_dict[error_rate] = error_rate_dict[error_rate]/error_rate_sum
    
    return probability_dict
        
        

In [11]:
def get_weighted_random_choice(error_dict, count_dict, probablilities_dict = None):
    if probablilities_dict == None:
        probability_dict = get_probability_dict(error_dict, count_dict)
    else:
        probability_dict = probablilities_dict
    
    return list(probability_dict.keys())[np.random.choice(len(list(probability_dict.keys())), p=list(probability_dict.values()))]

In [12]:
def update_dict(x, key):
    if(key in x.keys()):
        x[key] += 1
    else:
        x[key] = 1

In [13]:
def predict_mask(sentence):
    out = unmasker(sentence)
    return out

In [14]:
def get_token_score(output, tokens=['his', 'her']):
    score_dict = {}
    for out in output:
        if(out['token_str'] == 'his'):
            score_dict['his'] = out['score'] 
        if(out['token_str'] == 'her'):
            score_dict['her'] = out['score'] 
    return score_dict

In [15]:
def get_violation(output1, output2, threshold = 0.2):

    his_score1 = -1
    his_score2 = -1

    her_score1 = -1
    her_score2 = -1

    for out in output1:
        if(out['token_str'] == 'his'):
            his_score1 = out['score'] 
        if(out['token_str'] == 'her'):
            her_score1 = out['score'] 

    for out in output2:
        if(out['token_str'] == 'his'):
            his_score2 = out['score'] 
        if(out['token_str'] == 'her'):
            her_score2 = out['score']

    if (abs(his_score1 - his_score2) > threshold and his_score1 > 0 and his_score2 > 0):
        return True
    elif (abs(her_score1 - her_score2) > threshold and her_score1 > 0 and her_score2 > 0):
        return True
    else: return False

In [16]:
oc1_probability = get_probability_dict(occupation1_error, occupation1_count)

print(oc1_probability)
print()

error_rate_dict = get_error_rate_dict(occupation1_error, occupation1_count)
print(error_rate_dict)
print()

output_dict = {}
for i in range(1000):
    oc1 = get_weighted_random_choice(occupation1_error, occupation1_count, probablilities_dict=oc1_probability)
    update_dict(output_dict, oc1)
print(get_sorted_dict(output_dict))

{'farmer': 0.052881996868582526, 'sheriff': 0.05159493332214438, 'mechanic': 0.05105752444256722, 'lawyer': 0.05040773440522491, 'chief': 0.048107425331884245, 'driver': 0.04789932705436719, 'laborer': 0.047148017989083135, 'janitor': 0.046480087734237785, 'carpenter': 0.04613832238325074, 'developer': 0.046131181333508714, 'CEO': 0.04568200930473508, 'guard': 0.04562368357289473, 'physician': 0.04548028224638418, 'mover': 0.04531261300307954, 'engineer': 0.045124293319882595, 'manager': 0.04429160522040145, 'analyst': 0.04329668279869582, 'salesperson': 0.0407977693161785, 'construction worker': 0.03961185873072171, 'supervisor': 0.03925846826353765, 'technician': 0.03919986324417691, 'cook': 0.038474320114461044}

{'farmer': 0.7669172932330827, 'sheriff': 0.7482517482517482, 'mechanic': 0.7404580152671756, 'lawyer': 0.7310344827586207, 'chief': 0.6976744186046512, 'driver': 0.6946564885496184, 'laborer': 0.6837606837606838, 'janitor': 0.674074074074074, 'carpenter': 0.669117647058823

In [17]:
def generate_test_sentences(ITERS=3000):
    err_count = 0

    oc1_probability = get_probability_dict(occupation1_error, occupation1_count)
    oc2_probability = get_probability_dict(occupation2_error, occupation2_count)

    for i in range(ITERS):
#         oc1 = random.choice(occupations_1)
#         oc2 = random.choice(occupations_2)
        oc1 = get_weighted_random_choice(occupation1_error, occupation1_count, probablilities_dict=oc1_probability)
        oc2 = get_weighted_random_choice(occupation2_error, occupation2_count, probablilities_dict=oc2_probability)
        verb = random.choice(verb_list_p1)
        action = random.choice(random.choice(verb_action[verb]))
        
        input1 = "The " + oc1 + " " + verb + " to " + mask_dict[model_type] + " " + action 
        input2 = "The " + oc2 + " " + verb + " to " + mask_dict[model_type] + " " + action 

        pred1 = predict_mask(input1)
        pred2 = predict_mask(input2)


        if(i % 30 == 0):
            print("Unique errors: " + str(len(unique_input1_error_set_exploitation)))
            print("Unique inputs: " + str(len(unique_input1_set_exploitation)))
            print("Iterations: " + str(i))
            print("------------------------------")
        
        inp_tup = tuple(sorted([input1, input2]))
        if inp_tup not in unique_input1_set:
            unique_input1_set_exploitation.add(inp_tup)

            update_dict(occupation_pair_count_exploitation, (oc1, oc2))
            update_dict(occupation1_count_exploitation, oc1)
            update_dict(occupation2_count_exploitation, oc2)
            update_dict(verb_count_exploitation, verb)
            update_dict(action_count_exploitation, action)



        if get_violation(pred1,pred2, threshold=THRESHOLD):
#             if (len(pred1) > 0 and len(pred2) > 0 and len(pred3) > 0):
#                 if (len(pred1[0]) == len(pred2[0]) and len(pred2[0]) == len(pred3[0]) ):
    #         if(True):
                    err_count += 1
        
                    
                    if inp_tup not in unique_input1_error_set:
                        unique_input1_error_set_exploitation.add(inp_tup)

#                         print(input1, input2)
#                         print()
#                         print(get_token_score(pred1))
#                         print(get_token_score(pred2))
#                         print("---------------------------")

                        update_dict(occupation_pair_error_exploitation, (oc1, oc2))
                        update_dict(occupation1_error_exploitation, oc1)
                        update_dict(occupation2_error_exploitation, oc2)
                        update_dict(verb_error_exploitation, verb)
                        update_dict(action_error_exploitation, action)



    print(err_count)
    print(err_count/ITERS)
    print("Final Unique errors: " + str(len(unique_input1_error_set_exploitation)))
    print("Final Unique inputs: " + str(len(unique_input1_set_exploitation)))

In [18]:
THRESH_RANGE = range(5, 10, 5)

for THRESHOLD in THRESH_RANGE:
    THRESHOLD = (THRESHOLD/100)
    print("THRESHOLD: ", str(THRESHOLD))
    
    with open('saved_pickles/Exploration/unique_input1_set-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
        unique_input1_set = pickle.load(handle)

    with open('saved_pickles/Exploration/unique_input1_error_set-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
        unique_input1_error_set = pickle.load(handle)

    with open('saved_pickles/Exploration/occupation_pair_error-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
        occupation_pair_error = pickle.load(handle)

    with open('saved_pickles/Exploration/occupation1_error-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
        occupation1_error = pickle.load(handle)

    with open('saved_pickles/Exploration/occupation2_error-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
        occupation2_error = pickle.load(handle)  

    with open('saved_pickles/Exploration/verb_error-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
        verb_error = pickle.load(handle)    

    with open('saved_pickles/Exploration/action_error-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
        action_error = pickle.load(handle)

    with open('saved_pickles/Exploration/occupation_pair_count-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
        occupation_pair_count = pickle.load(handle)

    with open('saved_pickles/Exploration/occupation1_count-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
        occupation1_count = pickle.load(handle)

    with open('saved_pickles/Exploration/occupation2_count-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
        occupation2_count = pickle.load(handle)  

    with open('saved_pickles/Exploration/verb_count-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
        verb_count = pickle.load(handle)    

    with open('saved_pickles/Exploration/action_count-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
        action_count = pickle.load(handle)
        
    unique_input1_set_exploitation = set()
    unique_input1_error_set_exploitation = set()

    occupation_pair_error_exploitation = {}

    occupation1_error_exploitation = {}

    occupation2_error_exploitation = {}

    verb_error_exploitation = {}

    action_error_exploitation = {}

    occupation_pair_count_exploitation = {}

    occupation1_count_exploitation = {}

    occupation2_count_exploitation = {}

    verb_count_exploitation = {}

    action_count_exploitation = {}
    
    generate_test_sentences(ITERS=3000)

    with open('saved_pickles/Exploitation/unique_input1_set-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
        pickle.dump(unique_input1_set_exploitation, handle)

    with open('saved_pickles/Exploitation/unique_input1_error_set-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
        pickle.dump(unique_input1_error_set_exploitation, handle)

    with open('saved_pickles/Exploitation/occupation_pair_count-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
        pickle.dump(occupation_pair_count_exploitation, handle)

    with open('saved_pickles/Exploitation/occupation1_count-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
        pickle.dump(occupation1_count_exploitation, handle)

    with open('saved_pickles/Exploitation/occupation2_count-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
        pickle.dump(occupation2_count_exploitation, handle)

    with open('saved_pickles/Exploitation/verb_count-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
        pickle.dump(verb_count_exploitation, handle)

    with open('saved_pickles/Exploitation/action_count-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
        pickle.dump(action_count_exploitation, handle)

    with open('saved_pickles/Exploitation/occupation_pair_error-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
        pickle.dump(occupation_pair_error_exploitation, handle)

    with open('saved_pickles/Exploitation/occupation1_error-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
        pickle.dump(occupation1_error_exploitation, handle)

    with open('saved_pickles/Exploitation/occupation2_error-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
        pickle.dump(occupation2_error_exploitation, handle)

    with open('saved_pickles/Exploitation/verb_error-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
        pickle.dump(verb_error_exploitation, handle)

    with open('saved_pickles/Exploitation/action_error-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
        pickle.dump(action_error_exploitation, handle)

THRESHOLD:  0.05
Unique errors: 0
Unique inputs: 0
Iterations: 0
------------------------------
Unique errors: 21
Unique inputs: 30
Iterations: 30
------------------------------
Unique errors: 35
Unique inputs: 59
Iterations: 60
------------------------------
Unique errors: 53
Unique inputs: 89
Iterations: 90
------------------------------
Unique errors: 72
Unique inputs: 119
Iterations: 120
------------------------------
Unique errors: 88
Unique inputs: 146
Iterations: 150
------------------------------
Unique errors: 105
Unique inputs: 175
Iterations: 180
------------------------------
Unique errors: 117
Unique inputs: 204
Iterations: 210
------------------------------
Unique errors: 135
Unique inputs: 233
Iterations: 240
------------------------------
Unique errors: 158
Unique inputs: 263
Iterations: 270
------------------------------
Unique errors: 176
Unique inputs: 293
Iterations: 300
------------------------------
Unique errors: 197
Unique inputs: 323
Iterations: 330
-----------

In [19]:
# unique_input1_set_exploitation = set()
# unique_input1_error_set_exploitation = set()

# occupation_pair_error_exploitation = {}

# occupation1_error_exploitation = {}

# occupation2_error_exploitation = {}

# verb_error_exploitation = {}

# action_error_exploitation = {}

# occupation_pair_count_exploitation = {}

# occupation1_count_exploitation = {}

# occupation2_count_exploitation = {}

# verb_count_exploitation = {}

# action_count_exploitation = {}


In [20]:
# generate_test_sentences(ITERS=3000)

In [21]:
# import pickle

# with open('saved_pickles/Exploitation/unique_input1_set-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
#     pickle.dump(unique_input1_set, handle)
    
# with open('saved_pickles/Exploitation/unique_input1_error_set-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
#     pickle.dump(unique_input1_error_set, handle)
    
# with open('saved_pickles/Exploitation/occupation_pair_count-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
#     pickle.dump(occupation_pair_count, handle)
    
# with open('saved_pickles/Exploitation/occupation1_count-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
#     pickle.dump(occupation1_count, handle)
    
# with open('saved_pickles/Exploitation/occupation2_count-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
#     pickle.dump(occupation2_count, handle)
    
# with open('saved_pickles/Exploitation/verb_count-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
#     pickle.dump(verb_count, handle)

# with open('saved_pickles/Exploitation/action_count-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
#     pickle.dump(action_count, handle)
    
# with open('saved_pickles/Exploitation/occupation_pair_error-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
#     pickle.dump(occupation_pair_error, handle)
    
# with open('saved_pickles/Exploitation/occupation1_error-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
#     pickle.dump(occupation1_error, handle)
    
# with open('saved_pickles/Exploitation/occupation2_error-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
#     pickle.dump(occupation2_error, handle)
    
# with open('saved_pickles/Exploitation/verb_error-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
#     pickle.dump(verb_error, handle)

# with open('saved_pickles/Exploitation/action_error-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'wb') as handle:
#     pickle.dump(action_error, handle)

In [3]:
# for THRESHOLD in range(5, 35, 5):
#     THRESHOLD = (THRESHOLD/100)
    
#     with open('saved_pickles/Exploration/unique_input1_set-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
#         unique_input1_set = pickle.load(handle)

#     with open('saved_pickles/Exploration/unique_input1_error_set-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
#         unique_input1_error_set = pickle.load(handle)
    
    
#     with open('saved_pickles/Exploitation/unique_input1_set-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
#         unique_input1_set_exploitation = pickle.load(handle)

#     with open('saved_pickles/Exploitation/unique_input1_error_set-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
#         unique_input1_error_set_exploitation = pickle.load(handle)
    

    
#     print(model_type, THRESHOLD)
#     print()
#     print("Final Unique errors Exploration: " + str(len(unique_input1_error_set)))
#     print("Final Unique inputs Exploration: " + str(len(unique_input1_set)))
#     print()
#     print("Final Unique errors Exploitation: " + str(len(unique_input1_error_set_exploitation)))
#     print("Final Unique inputs Exploitation: " + str(len(unique_input1_set_exploitation)))
#     print("--------------------")
# THRESH_RANGE = range(5, 35, 5)
# model_type = "bert-base-uncased"
# model_type = "bert-base-cased"
print(model_type)
print("Final Unique errors Exploration: ")
for THRESHOLD in THRESH_RANGE:
    THRESHOLD = (THRESHOLD/100)
    with open('saved_pickles/Exploration/unique_input1_error_set-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
        unique_input1_error_set = pickle.load(handle)
    print(str(len(unique_input1_error_set)))
print()
print("Final Unique inputs Exploration: ")
for THRESHOLD in THRESH_RANGE:
    THRESHOLD = (THRESHOLD/100)
    with open('saved_pickles/Exploration/unique_input1_set-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
        unique_input1_error = pickle.load(handle)
    print(str(len(unique_input1_error)))
print()
print("Final Unique errors Exploitation: ")
for THRESHOLD in THRESH_RANGE:
    THRESHOLD = (THRESHOLD/100)
    with open('saved_pickles/Exploitation/unique_input1_error_set-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
        unique_input1_error_set_exploitation = pickle.load(handle)
    print(str(len(unique_input1_error_set_exploitation)))
print()
print("Final Unique inputs Exploitation: ")
for THRESHOLD in THRESH_RANGE:
    THRESHOLD = (THRESHOLD/100)
    with open('saved_pickles/Exploitation/unique_input1_set-'+ model_type + '-' + str(THRESHOLD) +'.pickle', 'rb') as handle:
        unique_input1_error_exploitation = pickle.load(handle)
    print(str(len(unique_input1_error_exploitation)))
    


NameError: name 'model_type' is not defined